# Obtain edges from a Pajek-like file

In [2]:
import time
import pandas as pd
import numpy as np
from nodes_to_pajek import *

In [2]:
s=time.time()
imax=100#000
for i in range(imax):
    for j in range(imax):
        a=1+1

print(time.time()-s)        

0.001239776611328125


##  Obtain full nodes file.  
Small example and Real case scenario

In [ ]:
FULL=True
if FULL==True:
    df_all=pd.read_json('hep_records.json',lines=True)
else:
    df_all=pd.read_json('nodes_small.json')
    


In [ ]:
dfr=df_all[df_all.references.map(len)>0].reset_index(drop=True)
dfr.shape

In [ ]:
pjt=nodes_to_pajek(dfr)

In [ ]:
pjt.shape

In [ ]:
pj=pjt.reset_index(drop=True)

In [10]:
pj.shape

(21733815, 2)

In [62]:
3/80*600000/60/24

15.625

## Reload and start here

In [3]:
#pj.to_json('pj_full.json')
pj=pd.read_json('pj_full.json')

In [4]:
pj_full=pj.copy()

In [5]:
pj=pj.sort_values(['references','recid']).reset_index(drop=True)

In [6]:
pj.shape

(21733815, 2)

In [7]:
#pj=pj[:4000074]#.reset_index(drop=True)

In [8]:
iini=0
iend=1000000
for i in range(1000):
    if i==0:
        ref_old=pj.loc[iend,'references']

    if pj.loc[iend,'references']!=ref_old:
        break
    iend=iend+i

pj=pj[iini:iend-1].reset_index(drop=True)
iini=iend-1    
iend=iend+1000000

##  Main code to find the edges

In [9]:
start_time=time.time()

DEBUG=True
if DEBUG: 
    #===log===
    f=open('kk.log','w')
    f.write('')
    f.close()
    #==DEBUG===========

id_col='recid'; refs_col='references'
# Half of the work is order by refs_col, and id_col
pj=pj.sort_values([refs_col,id_col]).reset_index(drop=True)
ed={}
t=()
new_ref=0
i=0
iprint=1000

while i<=pj.index[-1]:
    if not i%iprint:
        print(i,end='\r')
        if DEBUG:
            #===log===
            f=open('kk.log','a')
            f.write('{}\n'.format(i))
            f.close()
            #=======
    e1=pj.loc[i,id_col]
    #check next entry and beyond:
    for j in range(i+1,pj.index[-1]+1):
        #print('i,j',i,j) #,end='\r')
        e2=pj.loc[j,id_col]
        #DEBUG
        #if (e1,e2)==(190836,1644265):
        #    print('i,j,check',i,j)
        edges=False
        if pj.loc[i,refs_col]==pj.loc[j,refs_col]:
            edges=True
        #else:
            #print('newi',i,j)
            
        if edges:
            if e1<e2:
                t=(e1,e2)
                if not ed.get(t):
                    ed[t]=1
                    #DEBUG
                    #ed[t]=str( (i,j) )
                else: #increase the number of common refs_col
                    ed[t]=ed[t]+1
                    #DEBUG: 
                    #ed[t]=ed[t]+str(  (i,j)  )
        else:
            # Go to next i when nor additional edges are found
            break
    #search after j. 
    i=i+1 # Be sure to increase i to avoid infinite loop
    #if i>=25:
    #    break
    continue

print(time.time()-start_time)    

15287.510861873627


In [10]:
le=[{t:ed[t]} for t in ed.keys() if ed[t]>4 ]
len(le)

655851

import json

##### as requested in comment

with open('file.txt', 'w') as file:
     file.write(json.dumps({"le":le[0]})) 

In [24]:
ledf=pd.Series(ed)

In [25]:
ledf.shape

(308356360,)

In [27]:
ledf.to_json('ed.json')

In [28]:
ls -lh ed.json

-rw-r--r-- 1 restrepo restrepo 5.9G Apr 26 01:18 ed.json


In [53]:
ldf=pd.DataFrame()
ldf=ldf.append( ledf[:100000],ignore_index=True).transpose()

In [59]:
ldf[ldf[0]>10]

,0
"(625, 307561)",11.0


Continue in [run2](./run2/pajek.ipynb)

In [47]:
N=lambda x:  x*1052/100000/60
npt=1000000
print( N(npt),'m') 
print( N(npt)/60,'h') 

175.33333333333334 m
2.9222222222222225 h


In [48]:
le=[{t:ed[t]} for t in ed.keys() if ed[t]>4 ]
len(le)

3018

In [49]:
edf=pd.DataFrame({'tmp':le})
edf['edges'] =edf.tmp.map(lambda x: list(x.keys())[0]  )
edf['len_ed']=edf.tmp.map(lambda x: list(x.values())[0]  )
edf=edf.drop('tmp',axis='columns')
edf[:3]

,edges,len_ed
0,"(4380, 666911)",5
1,"(15239, 1387684)",5
2,"(98947, 129333)",5


In [50]:
#edf.to_json('example1E5.json')

Go to the previous code and run there. 

## Backup

IMPORTANT COMMITS: 
* 44fa444ba3cc1f9a052bf80051f44a0d11c5838a
  * Here the counting of the slow and fast to grouping mathc
